Wykożystaj dane z bazy 'bidevtestserver.database.windows.net'
||
|--|
|SalesLT.Customer|
|SalesLT.ProductModel|
|SalesLT.vProductModelCatalogDescription|
|SalesLT.ProductDescription|
|SalesLT.Product|
|SalesLT.ProductModelProductDescription|
|SalesLT.vProductAndDescription|
|SalesLT.ProductCategory|
|SalesLT.vGetAllCategories|
|SalesLT.Address|
|SalesLT.CustomerAddress|
|SalesLT.SalesOrderDetail|
|SalesLT.SalesOrderHeader|

In [0]:
# INFORMATION_SCHEMA.TABLES

jdbcHostname = "bidevtestserver.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "testdb"

tabela = spark.read \
  .format("jdbc") \
  .option("url",f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}') \
  .option("user","sqladmin") \
  .option("password","$3bFHs56&o123$") \
  .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("query","SELECT * FROM INFORMATION_SCHEMA.TABLES") \
  .load()
display(tabela)

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
testdb,sys,database_firewall_rules,VIEW
testdb,sys,ipv6_database_firewall_rules,VIEW
testdb,dbo,stg_tablename,BASE TABLE
testdb,SalesLT,Customer,BASE TABLE
testdb,SalesLT,ProductModel,BASE TABLE
testdb,SalesLT,vProductModelCatalogDescription,VIEW
testdb,SalesLT,ProductDescription,BASE TABLE
testdb,SalesLT,Product,BASE TABLE
testdb,SalesLT,ProductModelProductDescription,BASE TABLE
testdb,SalesLT,vProductAndDescription,VIEW


1. Pobierz wszystkie tabele z schematu SalesLt i zapisz lokalnie bez modyfikacji w formacie delta

In [0]:
from pyspark.sql.functions import col

In [0]:
jdbcHostname = "bidevtestserver.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "testdb"

tables = spark.read \
  .format("jdbc") \
  .option("url",f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}') \
  .option("user","sqladmin") \
  .option("password","$3bFHs56&o123$") \
  .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("query","SELECT * FROM INFORMATION_SCHEMA.TABLES ") \
  .load() \
  .where(col('TABLE_SCHEMA') =='SalesLT') \
  .toPandas()['TABLE_NAME'].values


tableDir = {}


for tableName in tables:
    tableContent = spark.read \
      .format("jdbc") \
      .option("url",f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}') \
      .option("user","sqladmin") \
      .option("password","$3bFHs56&o123$") \
      .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      .option("query",f"SELECT * FROM SalesLT.\"{tableName}\"") \
      .load() 
    tableDir[tableName] = tableContent
    tableContent.write.format('delta').save(f'dbfs:/FileStorage/lab4/tables/{tableName}')



Uzycie Nulls, fill, drop, replace, i agg
 * W każdej z tabel sprawdź ile jest nulls w rzędach i kolumnach
 * Użyj funkcji fill żeby dodać wartości nie występujące w kolumnach dla wszystkich tabel z null
 * Użyj funkcji drop żeby usunąć nulle, 
 * wybierz 3 dowolne funkcje agregujące i policz dla TaxAmt, Freight, [SalesLT].[SalesOrderHeader]
 * Użyj tabeli [SalesLT].[Product] i pogrupuj według ProductModelId, Color i ProductCategoryID i wylicz 3 wybrane funkcje agg() 
   - Użyj conajmniej dwóch overloaded funkcji agregujących np z (Map)

In [0]:
from pyspark.sql.functions import count, when, min
import numpy as np
from datetime import datetime 

In [0]:
for name, table in tableDir.items():
    print('-----------------------------------------------------------')
    print(name)
    print()
    print(table.select([count(when(col(c).isNull(),c)).alias(c) for c in table.columns]).collect())
    non_exist = {c: 'nonExistingValue' if t =='string' else 0 if  t =='timestamp' else int(np.array(table.agg(min(col(c))).collect())[0][0]-1)  for c, t in table.dtypes if np.array(table.select(count(when(col(c).isNull(),c))).collect()[0])[0] >0}
    table.na.fill(non_exist).show()
    table.na.drop().show()
    


-----------------------------------------------------------
Customer

[Row(CustomerID=0, NameStyle=0, Title=7, FirstName=0, MiddleName=343, LastName=0, Suffix=799, CompanyName=0, SalesPerson=0, EmailAddress=0, Phone=0, PasswordHash=0, PasswordSalt=0, rowguid=0, ModifiedDate=0)]
+----------+---------+----------------+-----------+----------------+----------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+--------------------+-------------------+
|CustomerID|NameStyle|           Title|  FirstName|      MiddleName|  LastName|          Suffix|         CompanyName|         SalesPerson|        EmailAddress|              Phone|        PasswordHash|PasswordSalt|             rowguid|       ModifiedDate|
+----------+---------+----------------+-----------+----------------+----------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+-----------

In [0]:

np.array(sales_order_header.select(count(when(col('SalesOrderID').isNull(),'SalesOrderID')).alias('SalesOrderID')).collect()[0])[0]

Out[118]: 0

In [0]:
sales_order_header = tableDir['SalesOrderHeader']

from pyspark.sql.functions import min,max, avg

sales_order_header.agg(min(col('TaxAmt')), max(col('TaxAmt')), avg(col('TaxAmt'))).show()
sales_order_header.agg(min(col('Freight')), max(col('Freight')), avg(col('Freight'))).show()

+-----------+-----------+-------------+
|min(TaxAmt)|max(TaxAmt)|  avg(TaxAmt)|
+-----------+-----------+-------------+
|     3.1163|  8684.9465|2163.58280313|
+-----------+-----------+-------------+

+------------+------------+------------+
|min(Freight)|max(Freight)|avg(Freight)|
+------------+------------+------------+
|      0.9738|   2714.0458|676.11962813|
+------------+------------+------------+



In [0]:
product = tableDir['Product']

product.groupBy('Color').agg(min(col('StandardCost')), max(col('StandardCost')), avg(col('StandardCost'))).show()

product.groupBy('ProductCategoryID').agg(min(col('StandardCost')), max(col('StandardCost')), avg(col('StandardCost'))).show()
product.groupBy('ProductModelID').agg(min(col('StandardCost')), max(col('StandardCost')), avg(col('StandardCost'))).show()

+------------+-----------------+-----------------+-----------------+
|       Color|min(StandardCost)|max(StandardCost)|avg(StandardCost)|
+------------+-----------------+-----------------+-----------------+
|        null|           0.8565|         101.8936|      23.55171000|
|       Multi|           6.9223|          38.4923|      34.03177500|
|      Silver|           8.9866|        1912.1544|     557.22356389|
|        Blue|          13.0863|        1481.9379|     567.15947692|
|       White|           3.3623|           3.3963|       3.37930000|
|       Black|           9.1593|        1898.0944|     434.11797978|
|      Yellow|          41.5723|        1481.9379|     597.43478056|
|         Red|          13.0863|        2171.2942|     858.17805526|
|        Grey|          51.5625|          51.5625|      51.56250000|
|Silver/Black|          17.9776|          35.9596|      28.42428571|
+------------+-----------------+-----------------+-----------------+

+-----------------+--------------

### EX2

In [0]:
from pyspark.sql.types import IntegerType, StringType

squareUDF = udf(lambda z: z**2, IntegerType())

cubeUDF = udf(lambda z: z**3, IntegerType())

centerTenUDF = udf(lambda z: z.center(10))

In [0]:
json = '\n'.join(spark.read.text('/mnt/brzydki.json').toPandas()['value'].T)


In [0]:
print(json)

{
   "type": "Collection",
   "jobDetails":
   {
      "jobId": 345345,
      "jobName": "XSDR_344985_WGZX_2022_A008_V",
      "jobType": "Stereo",
      "workspaceId": 345353,
      "changesTimestamp": "2022-08-06T03:29:46Z"
   },
   "maiaExtractProcessDetails":
   {
      "extractorVersion": "1.10",
      "extractionTimestamp": "2022-08-06T03:30:35Z",
      "extractMode": "Incremental Update",
      "maiaSavepoint": null,
      "queryBox": null,
      "fileSequenceNumber": 110572
   },
   "features": [
   {
      "type": "Feature",
      "geometry":
      {
         "type": "Polygon",
         "coordinates":
         [
            [
               [445751.000, 244970.000, 138.984], [446492.041, 244970.000, 101.861], [446492.041, 245088.372, 102.604],
               [445751.000, 245088.372, 140.312], [445751.000, 244970.000, 138.984]
            ]
         ]
      },
      "properties":
      {
         "featureGroup": "Named Extent+Group+None",
         "featureId": "a6a7e7ff-71d3-e4

In [0]:
from pyspark.sql.functions import from_json,schema_of_json

In [0]:
schema_of_json(json)

Out[192]: Column<'schema_of_json({
   "type": "Collection",
   "jobDetails":
   {
      "jobId": 345345,
      "jobName": "XSDR_344985_WGZX_2022_A008_V",
      "jobType": "Stereo",
      "workspaceId": 345353,
      "changesTimestamp": "2022-08-06T03:29:46Z"
   },
   "maiaExtractProcessDetails":
   {
      "extractorVersion": "1.10",
      "extractionTimestamp": "2022-08-06T03:30:35Z",
      "extractMode": "Incremental Update",
      "maiaSavepoint": null,
      "queryBox": null,
      "fileSequenceNumber": 110572
   },
   "features": [
   {
      "type": "Feature",
      "geometry":
      {
         "type": "Polygon",
         "coordinates":
         [
            [
               [445751.000, 244970.000, 138.984], [446492.041, 244970.000, 101.861], [446492.041, 245088.372, 102.604],
               [445751.000, 245088.372, 140.312], [445751.000, 244970.000, 138.984]
            ]
         ]
      },
      "properties":
      {
         "featureGroup": "Named Extent+Group+None",
      